# Introduction to LangChain v0.2.0 and LCEL: LangChain Powered RAG

In the following notebook we're going to focus on learning how to navigate and build useful applications using LangChain, specifically LCEL, and how to integrate different APIs together into a coherent RAG application!

In the notebook, you'll complete the following Tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Initialize a Simple Chain using LCEL
  4. Implement Naive RAG using LCEL
  
Let's get started!



# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

One of the [key features](https://blog.langchain.dev/langchain-v02-leap-to-stability/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages and added stability.

Instead of one all encompassing Python package - LangChain has a `core` package and a number of additional supplementary packages.

We'll start by grabbing all of our LangChain related packages!

In [4]:
!pip install -qU langchain langchain-core langchain-community langchain-openai

Now we can get our Qdrant dependencies!

In [5]:
!pip install -qU qdrant-client

Let's finally get `tiktoken` and `pymupdf` so we can leverage them later on!

In [6]:
!pip install -qU tiktoken pymupdf

## Task 2: Set Environment Variables

We'll be leveraging OpenAI's suite of APIs - so we'll set our `OPENAI_API_KEY` `env` variable here!

In [7]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Task 3: Initialize a Simple Chain using LCEL

The first thing we'll do is familiarize ourselves with LCEL and the specific ins and outs of how we can use it!

### LLM Orchestration Tool (LangChain)

Let's dive right into [LangChain](https://www.langchain.com/)!

The first thing we want to do is create an object that lets us access OpenAI's `gpt-4o` model.

In [8]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o")

<div style="border: 2px solid white; background: black; padding: 10px;">

#### ❓Question #1:

What other models could we use, and how would the above code change?

> HINT: Check out [this page](https://platform.openai.com/docs/models) to find the answer!

### Answer #1:

We could use any model that is hosted by OpenAI and available through the OpenAI API. The model needs to support the LangChain ChatOpenAI.

To implement the change, we would replace the string assigned to the model.

Example models include:
- GPT-3.5-Turbo (gpt-3.5-turbo)
- GPT-4 (gpt-4)
- Davinci-003 (text-davinci-003)
- Ada-002 ("text-ada-002")

We could also use other Providers with LangChain. This would change the wrapper class.
So for instance:
- CohereChat(model="cohere-xyz")
- HuggingFaceChat(model="facebook/blenderbot-400M-distill")

</div>

### Prompt Template

Now, we'll set up a prompt template - more specifically a `ChatPromptTemplate`. This will let us build a prompt we can modify when we call our LLM!

Note the {content} will be replaced when we run the chain

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are a legendary and mythical Wizard. You speak in riddles and make obscure and pun-filled references to exotic cheeses."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

### Our First Chain

Now we can set up our first chain!

A chain is simply two components that feed directly into eachother in a sequential fashion!

You'll notice that we're using the pipe operator `|` to connect our `chat_prompt` to our `llm`.

This is a simplified method of creating chains and it leverages the LangChain Expression Language, or LCEL.

You can read more about it [here](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel), but there a few features we should be aware of out of the box (taken directly from LangChain's documentation linked above):

- **Async, Batch, and Streaming Support** Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

- **Fallbacks** The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

- **Parallelism** Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

In the following code cell we have two components:

- `chat_prompt`, which is a formattable `ChatPromptTemplate` that contains a system message and a human message.
- `openai_chat_model`, which is a LangChain Runnable wrapped OpenAI client.

We'd like to be able to pass our own `content` (as found in our `human_template`) and then have the resulting message pair sent to our model and responded to!

In [10]:
chain = chat_prompt | openai_chat_model

Notice the pattern here:

We invoke our chain with the `dict` `{"content" : "Hello world!"}`.

It enters our chain:

`{"content" : "Hello world!"}` -> `invoke()` -> `chat_prompt`

Our `chat_prompt` returns a `PromptValue`, which is the formatted prompt. We then "pipe" the output of our `chat_prompt` into our `llm`.

`PromptValue` -> `|` -> `llm`

Our `llm` then takes the list of messages and provides an output which is return as a `str`!







In [11]:
print(chain.invoke({"content": "Hello world!"}))

content='Ah, a greeting as fresh as a wheel of Camembert! The world is but a stage, and we are mere players in a dairy delight of destiny. Tell me, seeker of truths, what curdling question churns within your soul today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 38, 'total_tokens': 90}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None} id='run-bc69cd63-6334-4053-bffb-82c78e079c88-0' usage_metadata={'input_tokens': 38, 'output_tokens': 52, 'total_tokens': 90}


Let's try it out with some different prompts!

In [12]:
chain.invoke({"content" : "Could I please have some advice on how to become a better Python Programmer?"})

AIMessage(content="Ah, seeker of serpentine syntax and looping logic! To become a wizard of the Pythonic arts, heed this curdled counsel:\n\n1. **Embrace the Zen of Python**: Like the wisdom hidden within a wheel of Parmigiano-Reggiano, the Zen of Python reveals truths both simple and profound. Seek out its aphorisms, for they shall guide your code to be as clean as a freshly cut slice of feta.\n\n2. **Practice, Practice, Practice**: Just as a fine Camembert needs time to mature, so too does your skill with Python. Write code daily, solve problems, and build projects. Let your experience age and ripen.\n\n3. **Read Others' Code**: Dive into the briny depths of open-source repositories. Examine the craftsmanship of seasoned coders, much like a connoisseur samples the complex layers of a Roquefort. Learn their tricks, patterns, and idioms.\n\n4. **Master the Libraries**: Python's ecosystem is as diverse as the world's cheese platter. From NumPy to Pandas, from Django to Flask, each libra

In [13]:
chain.invoke({"content" : "Answering icelandic, what is the icelandic favorite icecream? (Translatee into English in parentheses)"})

AIMessage(content='Ah, seeker of frosty delights, you venture into the realm of the North! There, amidst the land of fire and ice, lies the answer cloaked in creamy whispers and dairy dreams. The Icelandic tongue may reveal their cherished icy treasure as "Ís" (Ice Cream), but delve deeper, and you shall find "Gamli Ísinn" (The Old Ice Cream) or perhaps "Bragðarefur" (Taste Fox), a treat as cunning and varied as the flavors it holds. \n\nBut beware, for the true essence of their favorite lies in the mix of textures and tastes, much like the elusive "Brie of the Fjords"—smooth yet with a hint of the wild!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 59, 'total_tokens': 204}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-873e1f56-1619-4d16-9568-122b218d60c7-0', usage_metadata={'input_tokens': 59, 'output_tokens': 145, 'total_tokens': 204}

Notice how we specifically referenced our `content` format option!

Now that we have the basics set up - let's see what we mean by "Retrieval Augmented" Generation.

## Naive RAG - Manually adding context through the Prompt Template

Let's look at how our model performs at a simple task - defining what LangChain is!

We'll redo some of our previous work to change the `system_template` to be less...verbose.

We will also use a example from a Wargame called Epic Warpath which is currently in alpha testing and not available on the market

In [14]:
system_template = "You are a helpful assistant."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

chat_chain = chat_prompt | openai_chat_model
print(chat_chain.invoke({"content" : "Please define LangChain."}))
print("")
print("Epic Warpath")
print(chat_chain.invoke({"content" : "Please explain Epic Warpath"}).content)

content='LangChain is a framework designed to facilitate the development of applications powered by large language models (LLMs). It provides modular components and abstractions for essential elements such as prompt management, memory, and document retrieval. LangChain is particularly useful for creating a wide range of applications, including those that involve chatbots, Generative Question-Answering (GQA), summarization, and more. By offering a structured approach to integrating LLMs, LangChain helps developers build sophisticated applications that can leverage the full potential of language models.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 22, 'total_tokens': 128}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None} id='run-541a923a-eb2a-49aa-8df7-a0dbf184d145-0' usage_metadata={'input_tokens': 22, 'output_tokens': 106, 'total_tokens': 128}

Epic 

Well, that's not very good - is it!

The model does not know about Epic Warpath

The issue at play here is that our model was not trained on the idea of "LangChain", and so it's left with nothing but a guess - definitely not what we want the answer to be!

Let's ask another simple LangChain question!

In [15]:
print(chat_chain.invoke({"content" : "What is LangChain Expression Language (LECL)?"}).content)

LangChain Expression Language (LECL) is a domain-specific language designed to facilitate the construction, querying, and manipulation of data within the LangChain ecosystem. LangChain is an open-source framework for developing applications powered by language models, and LECL plays a critical role in enabling developers to efficiently interact with these models and the data they process.

LECL allows developers to write expressions that can be used to filter, transform, and analyze data in a concise and expressive manner. This is particularly useful for tasks such as managing complex data pipelines, performing advanced queries, and implementing custom logic within LangChain applications.

Key features of LECL include:

1. **Simplicity and Expressiveness**: LECL is designed to be easy to learn and use, with a syntax that is both concise and powerful, allowing developers to express complex operations in a straightforward manner.
2. **Integration with LangChain**: LECL is tightly integra

While it provides a confident response, that response is entirely ficticious! Not a great look, OpenAI!

However, let's see what happens when we rework our prompts - and we add the content from the docs to our prompt as context.

In [16]:
HUMAN_TEMPLATE = """
#CONTEXT:
{context}

QUERY:
{query}

Use the provided context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, respond with "I don't know"
"""

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"query" : "What is LangChain Expression Language?", "context" : CONTEXT}).content)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. It provides several benefits, including full sync, async, batch, and streaming support, the ability to handle errors gracefully with fallbacks, automatic parallelism for components that can run in parallel, and seamless integration with LangSmith for maximal observability and debuggability.


In [17]:
HUMAN_TEMPLATE = """
#CONTEXT:
{context}

QUERY:
{query}

Use the provided context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, respond with "I don't know"
"""

CONTEXT = """
WELCOME TO EPIC WARPATH
Set in and around the vast Galactic Co-Prosperity Sphere, Epic Warpath is an immersive and exciting game of futuristic battles fought on alien worlds.
All of this is represented by collections of highly detailed Mantic models and is played out on your tabletop with evocative terrain, special dice and tokens to keep track of the action.
A game of Epic Warpath is usually played between two players, each controlling an army of highly trained (or degenerate) warriors and futuristic vehicles, all vying for control of strategic objectives.
Games involving more players and/or more armies are possible, but they require additional rules and so will be treated separately in separate expansions, articles, etc.
 """

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"query" : "What is Epic Warpath?", "context" : CONTEXT}).content)

Epic Warpath is an immersive and exciting tabletop game of futuristic battles fought on alien worlds, set in and around the vast Galactic Co-Prosperity Sphere. Players use collections of highly detailed Mantic models, along with evocative terrain, special dice, and tokens to keep track of the action. Typically played between two players, each controls an army of warriors and futuristic vehicles, competing for control of strategic objectives. Games with more players or armies require additional rules, which are covered in separate expansions and articles.


You'll notice that the response is much better this time. Not only does it answer the question well - but there's no trace of confabulation (hallucination) at all!

> NOTE: While RAG is an effective strategy to *help* ground LLMs, it is not nearly 100% effective. You will still need to ensure your responses are factual through some other processes

That, in essence, is the idea of RAG. We provide the model with context to answer our queries - and rely on it to translate the potentially lengthy and difficult to parse context into a natural language answer!

However, manually providing context is not scalable - and doesn't really offer any benefit.

Enter: Retrieval Pipelines.

## Task #4: Implement Naive RAG using LCEL

Now we can make a naive RAG application that will help us bridge the gap between our Pythonic implementation and a fully LangChain powered solution!

## Putting the R in RAG: Retrieval 101

In order to make our RAG system useful, we need a way to provide context that is most likely to answer our user's query to the LLM as additional context.

Let's tackle an immediate problem first: The Context Window.

All (most) LLMs have a limited context window which is typically measured in tokens. This window is an upper bound of how much stuff we can stuff in the model's input at a time.

Let's say we want to work off of a relatively large piece of source data - like the Ultimate Hitchhiker's Guide to the Galaxy. All 898 pages of it!

> NOTE: It is recommended you do not run the following cells, they are purely for demonstrative purposes.

The following has a section from the rules of a wargame that is currently being playtested and is not released to market

In [18]:
CONTEXT = """
Bases and Units
Groups of models that are selected within an army and which move and fight together are called Units. All the models within a Unit will be mounted on one or more Bases.
When the rules refer to a Unit, it means all the Bases within that Unit.
Unit Profiles
All Units in Epic Warpath have a profile of statistics or ‘stats’ which show how effective the Bases in the Unit are at moving, shooting, assaulting and surviving in a combat zone. In addition, the height of the Bases in a Unit, their Keywords and weaponry are all included in the profile.
Some Units may be so poor at doing something they will be effectively unable to do it at all! Such Units will have a stat value of “-“ where appropriate.
Some examples of Unit profiles are shown below. [EXAMPLE PROFILES, ANNOTATED]
Version PT v1
5
A Unit profile details the following information:
Unit Name: Each Unit is identified by its title.
Type: The type of Unit is used for a number of rules in the game. Types include Infantry, Heavy Infantry, Vehicle, Bike, Walker and Super-heavy. Units listed as Command, will also have a type in Brackets. For example Command (Infantry), or Command (Vehicle). For the purposes of the rules, they are both types.
<add pictures showing examples of unit types>
Speed (SP): How far a Unit can move in inches, given as two values (n/n – Advance/Sprint). The first number is used to represent the Unit’s basic movement. The second number is used when the Unit Sprints.
Shoot (SH): The ability of a Unit to fire ranged weapons accurately. A Unit with a stat value of – cannot Shoot.
Assault (AS): The skill or strength of a Unit in hand-to-hand combat. If a Unit has a stat value of – it cannot initiate an Assault and will roll no dice if assaulted.
Armour (AR): The armour, or resilience of a model to taking damage.
Height (HT): The Unit’s Height value. This value is used primarily to determine Line of Sight and Cover.
Unit Strength (US): This value represents the ability of the Unit to take and hold an Objective. Units with higher US values will win out if an Objective is contested. Typically, Infantry will have better US than bikes or vehicles, representing their ability to bed in and lock down an area. The value is usually expressed as n/n (e.g. 4/2 or 3/1).
Nerve Tests
Nerve is a single value that is set for an army as a whole, instead of an individual Unit. If a Unit has to make a Nerve test for any reason, roll a single D8 and compare the result to the Nerve value of the army. If the result equals or beats the army Nerve value, the test is passed. Otherwise, it is a failure.
Points (PTS): The cost to purchase the Unit when building your army. The points cost includes a number of Bases and standard equipment. PTS cost is also used to determine the winner in some Scenarios.
Weapon Stats: The ranged and assault weaponry of the Bases within the Unit, and their Keywords. Where Attack Dice are listed, this is the number of dice that a weapon gets for each Base in the Unit.
Keywords: Some Units and weapons will have special abilities that allow them certain advantages, or disadvantages, in the game. These Keywords may be for the Unit as a whole or for individual weapons within the Unit. This will be clearly shown in the Unit profile. For example, in the sample profile opposite, all the models in the Unit have the XXX TBC
Bases and Units
All models in Epic Warpath, except some vehicles, are mounted on round bases. A Base could represent a single walker or a small squad of infantry models, for example.
One or more Bases are formed into Units. Bases in a Unit operate together during the game.
Vehicles without bases.
If a vehicle is supplied without a base, consider the footprint of its hull to be its “Base”.
Some people prefer to create their own bases for these vehicles. This is fine, but such scratch-built bases should not be much larger than the vehicle itself. If players have a mix of based and non-based vehicles, ignore the vehicle bases for the purposes of the rules.
Base Contact
A Base is in base contact with something when it is physically touching it (such as an enemy base, building, Objective etc.). When two Bases are touching, they are in base contact.
Version PT v1
6
Armies
In a game of Epic Warpath you and your opponent each take control of an Army of soldiers to play an exciting battle game. Once the battlefield has been set up with terrain and any objectives, your chosen Scenario will tell you how to deploy your Units and how to achieve victory!
Friendly Elements
If a rule refers to friendly Units/Bases, this means all Units and Bases in your army.
Enemy Elements
If a rule refers to enemy Units/Bases, this means all Units and Bases in your opponent’s army.
Neutral Elements
If a Scenario requires the use of third party elements, these are considered neutral and are classed as enemy elements for both players.
In Play
All Units and Bases are In Play if they are currently placed on the gaming table, including Flying Units, and also includes Units within transports (even if the actual models are currently placed to one side until they emerge from their vehicle). Units and Bases that are currently in reserve (awaiting deployment), or have been destroyed, are not In Play.
Measuring
A tape measure marked in inches is required to play Epic Warpath in order to determine how far Bases move or shoot etc.
Measurement between Bases, and range to targets is always taken as the distance between the closest points on the two elements involved.
You can measure to check distances or range at any time.
Line of Sight
Line of Sight (LOS) is determined by using a top-down view, looking down on the table as if from a drone or satellite. Often, whether Bases and Units have LOS to a target is obvious, but the top-down approach makes it clear when where is any doubt.
Bases have a 360 degree arc of sight – so they can see all around them.
To determine if a Unit has LOS to a target, draw a straight, imaginary line from any part of a Base in the friendly Unit to any Base in the target enemy Unit. Bases from the friendly Unit checking for LOS, or target Unit, and any Bases with the Fly Keyword, do not block LOS. Any Bases, or gaps between Bases, in other intervening Units will block LOS up to that Unit’s height.
Players should check for LOS from each Base in a Unit, as only some may be able to see a target (or not all the Bases in an enemy Unit may be visible).
If a Base checking for LOS can draw an uninterrupted line to all parts of an enemy Base, then LOS is considered to be Clear.
If it’s impossible to draw this imaginary line to any part of an enemy Base, without it being interrupted by terrain or Bases with a Height equal or greater than both target and the Base drawing LOS, then LOS is Blocked.
LOS from a Base to the enemy is considered Partially Blocked if:
•
The imaginary line can be drawn to some, but not all of the enemy base, because it is obscured by something that blocks LOS.
•
The line passes over a Base or terrain feature that is lower than both target and shooter (e.g. firing over an intervening wall or shorter Unit).
•
The line passes over a Base or terrain feature that is equal in height to either the target or shooter, and lower than the other.
An enemy Unit can be seen so long as LOS is either Clear or Partially Blocked. When LOS is Partially Blocked, the enemy Unit is often said to be In Cover.
Version PT v1
7
The benefit of Elevation
Units can ignore any intervening terrain elements or Units that are 3 or more Height levels lower than they are.
For example, a Unit in a solid building is Height 5 and so can ignore (see clearly over) intervening elements of Height 2 or less.
Large Targets
Conversely, targets that themselves are 3 Height levels or taller than intervening terrain or Units, cannot claim LOS is Blocked or Partially Blocked by those elements.
For example, a Height 2 Unit can ignore a Height 1 barricade when shooting up at a Unit in a Height 6 building.
Version PT v1
8
Version PT v1
9
Rounds, Phases and Turns
A game of Epic Warpath is played over a series of Rounds, which are each conducted in a number of Phases. Within each Phase each player will alternate taking Turns to activate their Units, one at a time, until both players have completed activating all the eligible Units in their army. Once one of the players has completed all their possible activations in a phase, the other player then completes all of theirs.
The Scenario being played will determine the number of Rounds in a game, together with terrain set-up, troops deployment, victory conditions and so on; see the Scenario Section on page XXX
Round sequence
Round Sequence
1.
Command Phase
2.
Movement Phase
3.
Overwatch Phase
4.
Combat Phase
5.
End Phase
Command Phase
In the Command Phase, players perform the following in order.
1.
Roll for Command Points
2.
Assign Action Tokens
3.
Roll for Initiative
4.
Reveal Hidden Action Tokens
Movement Phase
Units that have been assigned Sprint or Advance Action Tokens take it in turn to move in this Phase. Units which will engage the enemy in Assault, swap their Sprint token for a Combat Token, otherwise the token is simply removed once the Sprint is complete. Units with Advance tokens perform any movement and then swap them for Combat Tokens.
Overwatch Phase
Units with Overwatch Action Tokens take it in turns to shoot at the enemy using any ranged weapons they have. Once a Unit has fired, its token is removed.
Combat Phase
Units with Combat Tokens take it in turn to either fight in close combat (Assault) with enemy troops, or shoot with ranged weapons. As each Unit completes its action, its token is removed.
End Phase
Any special rules which take place in the End Phase are carried out. Players roll for Pin marker removal. Players then check the Scenario conditions to see if some has achieved victory. If not, the game continues and the next Round starts with the Command Phase.
Version PT v1
10
Command Phase
In the Command Phase, players perform the following in order.
Roll for Command Points
Each player rolls three Command Dice, plus any additional Command Dice generated by Tactician units (see page @@), to generate their own Pool of Command Points.
Assign Action Tokens
The three possible Action Tokens to place in this phase are:
Overwatch
Advance
Sprint
Mandatory Tokens
First of all, players place the following Tokens, face-up next to the relevant units, as described below.
•
Engaged Units
Units that are engaged in Assault with each other from a previous Round must be given Combat Tokens instead of one of the Action Tokens listed above.
•
Pinned Units
Unengaged units that start the Round with a Pin marker must be given an Advance Token. Engaged Units receive no tokens of any kind.
•
Command Units
Unengaged Command Units that are not Pinned must be given both a Sprint and an Overwatch Token.
Hidden Tokens
Simultaneously, both players select the Action Token they want for each remaining unit they have In Play that has not yet received a Token (i.e. that was not included in the Mandatory Tokens listed above).
These Action Tokens should be placed face down next to the Unit receiving the Token.
Note that units aboard transports are not assigned any Token, but are assumed to have the same Token as their transports (see page @@).
Roll for Initiative
Unless a Scenario instructs otherwise, both players roll a D8 in the first Command Phase and the winner has the Initiative for the Round. Re-roll ties.
In each subsequent Command Phase, players roll for Initiative again to see which one has it for that Round. If a tie is rolled, the player that didn’t have the Initiative in the previous Round wins.
The player with the Initiative for the Round should take the Initiative Token as a reminder.
The Initiative Token
For the rest of the Round, the player with the Initiative Token can decide which players goes first in each phase.
In addition, whenever two rules are triggered at the same time, the player with Initiative can decide in which order all of these simultaneously triggered rules are resolved (unless the rules state otherwise).
For example, imagine a unit is assaulting another and it has a rule like a Strategic order (see page @@), which can be triggered ‘when assaulting an enemy’. Similarly, the defender could have a rule that can be triggered ‘when assaulted’. Which player decides whether to activate their rule first? The player with initiative decides which player must trigger their rule first.
Version PT v1
11
Reveal Hidden Action Tokens
Both players turn over and reveal all the hidden Action Tokens they gave to their Units.
If a player has forgotten to place a Token on a Unit, it is now given a Combat Token.
Note that this is the last chance for a unit to receive Action Tokens. If you forget to do so and you start the Movement phase, then any forgotten Units will have no action for this Round.
FOG OF WAR
Once you and your opponent are familiar with the rules, you may wish to play using the “Fog of War” optional game mode.
Instead of revealing all the Action Tokens at once, each player only flips and reveals Tokens as they activate each unit (or as an Overwatch charge reaction). This means you can keep you opponent guessing as to your plans!
You can’t look at your opponent’s Tokens, but you can check your own at any time.
Any remaining Overwatch Tokens are revealed all together at the end of the Movement Phase.
Command Points & Orders
Soldiers will work best if properly led and organised which involves solid leadership and good orders. To simulate this, we use Command Points and Orders to represent your ability to get the most from your army.
Generating the Command Pool
Epic Warpath Command Dice
At the start of each Round both players create their Command Dice Pool by taking three BLACK Command Dice. Additional dice of the appropriate colour (BLACK, ORANGE or GREEN as shown on a Unit’s profile) are added to the roll for any of their Units In Play with the Tactician Keyword.
Note that the Tactician Keyword may be on some Units that are not Command Units, and these Units will contribute their Command Dice as normal but may not themselves issue orders (see below). A player’s Command Dice may be reduced as the game goes on if they are careless with their Commanders!
Each player rolls all of their Command Dice, and may immediately re-roll one Command Die for each Command Unit they have In Play.
After this is done, add up the Command Points shown on the dice. This is the total number of Command Points each player has available for the Round about to be played.
Keep aside the Command Dice you rolled and use them as counters to keep track of how many Command Points you have available that Round.
During a Round, Command Points are spent in a number of ways to increase the utility of an army. Players must either spend all their available Command Points for the Round or lose them. They cannot be saved for future Rounds.
As you spend your Command Points, rotate the Command Dice to reflect how many Command points you have left for the Round.
Version PT v1
12
Tactical Orders
The following Tactical Orders are available to both players during a Round by spending Command Points from their pool.
These orders can be used on any friendly unit that is In Play, as it is assumed that they are issued by squad commanders or vehicle squadrons’ commanders, which are imbedded in their unit.
Tactical Orders (available to all armies)
Command Point Cost Order Timing Effect
1
Keep Going!
(Extra Activation)
After activating a Unit in a Phase and completing its action
Activate another Unit before returning play to your opponent to have their turn. A player may only interrupt the normal turn sequence like this once before their opponent must take another turn.
This order may not be used with the first player’s first Activation in a Phase (i.e. the player going first in a Phase may not Activate two Units until their opponent has Activated at least one). 1 Fight on! (Re-roll) After rolling to hit or to damage a target Re-roll up to two dice in your roll that failed to hit or damage a target. Remember though that you cannot re-roll any dice that have already been re-rolled for other reasons (e.g. Marksman). You can only do this once per roll against a specific target (with all the involved weapons). For example, you roll eight dice and roll five misses. You can spend a Command point to re-roll two misses, but you cannot then decide that you are going to re-roll two more misses. When engaging multiple targets (in multiple assaults or when a unit is allowed to split fire by a special rule, or a Blast Template covers different targets…) you could for example roll three dice against target A, and spend a Command point to re-roll two misses. Then you roll five dice to hit against target B, and you could also spend another Command point to re-roll two misses.
Version PT v1
13
Strategic Orders
Each faction has Strategic Orders that are unique to their armies. So, for example, the Enforcers have access to different Strategic Orders than the Plague.
Like Tactical Orders, Strategic Orders are special effects that are paid for using Command Points.
All Strategic Orders must be declared before any relevant dice are rolled.
Unlike a Tactical Order, a Strategic Order comes from more senior commanders, so unless specified otherwise, it can be used only on friendly Units that are within the Command Radius (in inches) of a friendly Command Unit (as normal, at least one Base needs to be within range). Strategic Orders can be used on the Command Unit itself. Units that are out of the Command Radius of any friendly Command Units, cannot have any Strategic Orders played on them.
If a Command Unit is occupying a building or being transported by a vehicle (as explained in the relevant sections of these rules), measure the Command Radius to the building itself or any of the transport vehicles carrying the Command Unit. Likewise, to issue an order to a Unit inside a transport or a building, the Command Unit must be in range of the transports or the building.
Only one Strategic Order may be used each activation. The Strategic Order may be used before or after, but not during, an action – and each order will specify when it can be used. Most Strategic Orders will affect a single Unit and are resolved immediately, or will affect that Unit’s next action.
For Example: An Enforcer player wants to use the Go, Go, Go! Strategic Order on one of their units with a Sprint Action Token, that is within the 16” Command Radius of an Enforcer Captain.
To use this Strategic Order, the player pays the cost of 2 Command Points when the target unit activates. When the Unit has completed its movement, and if it is not engaged in base contact with the enemy, its Sprint Action Token is replaced with a Combat Token.
Simultaneous Rules
Sometimes two or more strategic orders, or indeed any other rules, may be triggered at the same time (e.g. when a unit moves within a certain range). If it matters in which order these rules are going to be resolved, then the player with the Initiative Token decides the order in which they are resolved (unless the rules state otherwise).
Version PT v1
14
Movement Phase
In the Movement Phase, only Units that have Sprint or Advance Tokens may activate and move. Units that have Overwatch Action Tokens are preparing to offering covering, aimed or snap-fire in support of their comrades later (or if they know they may be charged so they can fire as a reaction).
The player with the Initiative chooses to go first or second. Players then take it in alternating turns to “activate” and move their Units one at a time. Players cannot elect to pass their turn. Once a player runs out of Units to move, the other player finishes moving their remaining Units.
Coherency
Soldiers in Epic Warpath have been trained to act as part of a team following the directions of their Unit and squad leaders. They must move and fight together.
Each Base in a Unit must be within 1” of at least one other Base in the Unit, in an uninterrupted chain or group, and within 6” of all the other Bases in the Unit. This is called Coherency.
Removal of Bases from a Unit (e.g. due to casualties) should be performed to maintain Unit Coherency whenever possible.
If any Base from a Unit finds itself outside of Coherency, the first time the Unit moves it must return within Coherency.
Version PT v1
15
Moving Units
To move a Unit, simply measure the distance from the edge of the Bases and move them up to the maximum the Unit can move for the action being performed. This move can be in a straight line, or curved to avoid other models or terrain, and end up facing into any direction. Do make sure that the final positions of all the Bases in the Unit keeps it in Coherency and that no part of any base has moved more than the maximum distance allowed.
When a Unit with an Advance action moves, it may move up to the first of its Speed stat values. So, a Unit with a Speed of 4/8, could move up to 4”.
When a Unit with a Sprint action moves, it may move up to the value of its second Speed stat value. So, a Unit with a Speed of 4/8, could move up to 8”. Only Units with Sprint actions are permitted to engage enemy Bases in an Assault by moving in to base-to-base contact (see below).
A player may elect not to move a Unit at all when it activates.
Interpenetration of Units
No Unit’s bases may come within 1” of any enemy Unit unless they are engaging that Unit in an Assault (see below) by moving into base-to-base contact.
A Unit may move through another friendly Unit as long as its Bases end their moves at least 1” away.
Some very large vehicles can even just turn on their centre point and rotate through Friendly models, so long as they finish any turning clear of other Bases.
In general, friendly Units should, as much as possible, be kept at least 1” apart, and can never be in base contact with other friendly Units (except where the rules specify otherwise). If you cannot keep friendly units at least 1” apart, a smaller distance is acceptable, but make sure it is clear to your opponent which Bases belong to which Unit.
Moving Units onto the Table
Some Units may enter the table after deployment as part of Scenario rules or by bringing in Reserves. When they are ready to enter play, Units in reserve must be given an Advance or Sprint Token in the Command Phase. To move onto the table simply activate the Unit and then measure from the edge of the board onto the table as part of their Advance or Sprint action, making sure its Bases end in Coherency as normal. The Unit is now In Play. A Unit coming onto the table in this way may not Assault an enemy Unit in the same Round that it arrives, even with a Sprint Token.
If you are playing the Fog of War game variant, Units in Reserve may be given an Overwatch Token as a “bluff” to fool your opponent, but then will not actually enter play that Round!
Version PT v1
16
Moving Units off of the Table
No Unit may move off the table under normal circumstances and must immediately stop at the edge of the table if forced to do so. If permitted to leave the table by Scenario-specific rules then all Bases must be able to reach the table edge during its movement to do so. If a Unit leaves the table, simply remove all the Bases from the Unit and place them off the table, making it clear they have not been destroyed. They are, however, no longer In Play.
Assaults - Engaging the enemy
A Unit with a Sprint token may move into base-to-base contact with a single enemy Unit, to engage them in an Assault. Simply move each assaulting base into base contact with a single base in the target unit, trying to engage as many enemy Bases as possible (normal movement rules permitting), while still keeping the Assaulting unit in coherency.
In this example, the Peacekeepers (PK) have a Sprint Action Token and want to move to Assault the Zombies. Each PK Base can move up to 8” when Sprinting, so only three are able to reach and make base contact with as many of the Zombies as they can. The other Bases in the Unit will move as far as they can to try and support the attack and remaining in Coherency.
Version PT v1
17
While engaged in a multiple combat involving several Units, the 1” rule for keeping Units separate does not apply to the Units involved.
After engaging the enemy as described above, Assaulting Units swap their Action Token(s) for a Combat Token.
However, if any part of the moving Unit’s movement (even a single Base) was in or through Difficult terrain, or into or from a Building, they are Hindered and their Combat Token is swapped for a Combat (-1) token instead.
Any Unit that is Assaulted and therefore engaged in base-to-base contact by the enemy, swaps any Sprint or Advance tokens it still has for a Combat Token (losing its chance to move!).
Vehicles and Walkers
If a Vehicle/Walker Unit is assaulted by anything other than another Super-heavy/Vehicle/Walker while they still have an Advance or Sprint Token, they retain their Advance/Sprint Token.
When activated, they can move out of base contact with the unit(s) that engaged them. If they can’t (or choose not to) move out of base contact when activated, they swap their Token for a Combat Token.
Super-heavy Units
If a Super-heavy Unit is assaulted by anything other than another Super-heavy Unit while they still have an Advance or Sprint Token, they retain their Advance/Sprint Token.
When activated, they can move out of base contact with the unit(s) that engaged them. If they can’t (or choose not to) move out of base contact when activated, they swap their Token for a Combat Token.
Charging a Unit on Overwatch
If a unit assaults a unit that has an Overwatch Token, move the assaulting Bases in as normal, then immediately resolve the Overwatch fire against the assaulting unit (see page @@) and finally remove the Overwatch Token from the unit.
Version PT v1
18
End of activation
After a unit activates, removing or swapping tokens as described below helps keep track of which Units each player has activated, and also acts as reminders for future phases and actions.
After a unit is activated:
•
If it has an Advance Token, swap it for a Combat Token.
Combat Token
•
If it has a Sprint Token, but didn’t engage the enemy in base-to-base contact, simply remove its Token.
•
If it has a Sprint Token and engaged the enemy in an Assault (base-to-base contact) then swap the Sprint Token for a Combat Token, or a Combat (-1) Token if Hindered, as described above.
Combat (-1) Token
Using a Combat (-1) Token is simply a helpful reminder that the Unit was Hindered when making its Sprint into the enemy. Both types of Token are Combat Tokens.
Command Units
If Command Units move and don’t engage the enemy in base-to-base contact, they discard their Sprint Token, but keep their Overwatch Token.
If Command Units assault an enemy unit, they swap both of their Tokens for a Combat Token.
If they are assaulted, they lose both of their Tokens, but immediately resolve the Overwatch Action against the assaulting unit.

Reserves
Units placed in Reserve, can only move onto the table from Round 2 onwards. The exact Round they enter is up to the player. See Moving onto the table on page @@.
In the Movement phase, you can choose to activate a Unit in Reserve instead of a Unit already on the table, so that the Unit in reserve enters the game, as described on page @@.
Units coming in from Reserve during Round 2 can enter the game from any point on a board edge included within the player’s own Deployment Zone(s).
Units coming in from Reserve during Round 3 or later can enter the game from any point along the entire board edge that includes any part of the player’s Deployment Zone(s).
Note that not all Scenarios permit the use of Reserves.
Starting the game!
Once deployment is done, or if the Scenario Parameter does not require deployment at all and all forces start the game in Reserve, you are now ready to begin Round 1 of the game.
"""

We can leverage our tokenizer to count the number of tokens for us!

In [19]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

len(enc.encode(CONTEXT))

6022

The full set comes in at a whopping *636,144* tokens. In our case 6022.

So, we have too much context. What can we do?

Well, the first thing that might enter your mind is: "Use a model with more context window", and we could definitely do that! However, even `gpt-4-128k` wouldn't be able to fit that whole text in the context window at once.

So, we can try splitting our document up into little pieces - that way, we can avoid providing too much context.

We have another problem now.

If we split our document up into little pieces, and we can't put all of them in the prompt. How do we decide which to include in the prompt?!

> NOTE: Content splitting/chunking strategies are an active area of research and iterative developement. There is no "one size fits all" approach to chunking/splitting at this moment. Use your best judgement to determine chunking strategies!

In order to conceptualize the following processes - let's create a toy context set!

### TextSplitting aka Chunking

We'll use the `RecursiveCharacterTextSplitter` to create our toy example.

It will split based on the following rules:

- Each chunk has a maximum size of 100 tokens
- It will try and split first on the `\n\n` character, then on the `\n`, then on the `<SPACE>` character, and finally it will split on individual tokens.

Let's implement it and see the results!

In [21]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = tiktoken_len,
)

chunks = text_splitter.split_text(CONTEXT)

len(chunks)

81

In [22]:
for chunk in chunks:
  print(chunk)
  print("----")

Bases and Units
Groups of models that are selected within an army and which move and fight together are called Units. All the models within a Unit will be mounted on one or more Bases.
When the rules refer to a Unit, it means all the Bases within that Unit.
Unit Profiles
----
Unit Profiles
All Units in Epic Warpath have a profile of statistics or ‘stats’ which show how effective the Bases in the Unit are at moving, shooting, assaulting and surviving in a combat zone. In addition, the height of the Bases in a Unit, their Keywords and weaponry are all included in the profile.
----
Some Units may be so poor at doing something they will be effectively unable to do it at all! Such Units will have a stat value of “-“ where appropriate.
Some examples of Unit profiles are shown below. [EXAMPLE PROFILES, ANNOTATED]
Version PT v1
5
A Unit profile details the following information:
Unit Name: Each Unit is identified by its title.
----
Unit Name: Each Unit is identified by its title.
Type: The typ

As is shown in our result, we've split each section into 100 token chunks - cleanly separated by `\n\n` characters!

<div style="border: 2px solid white; background: black; padding: 10px;">

#### 🏗️ Activity #1:

While there's nothing specifically wrong with the chunking method used above - it is a naive approach that is not sensitive to specific data formats.

Brainstorm some ideas that would split large single documents into smaller documents.

1. Split based on document structure such as:
    1. Chapters or sections
    2. Paragraphs and/or sentences
    3. Maintain list structures
2. Sliding window with an overlap
3. If its a group of documents eg emails or blogs, split it by individual emails or posts
4. Split based on time if a history of events
5. If questions and answers, keep them together
6. Keep like themes together

</div>

## Embeddings and Dense Vector Search

Now that we have our individual chunks, we need a system to correctly select the relevant pieces of information to answer our query.

This sounds like a perfect job for embeddings!

We'll be using OpenAI's `text-embedding-3` model as our embedding model today!

Let's load it up through LangChain.

In [23]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

<div style="border: 2px solid white; background: black; padding: 10px;">

#### ❓ Question #2:

What is the embedding dimension, given that we're using `text-embedding-3-small`?

> HINT: Check out the [docs](https://platform.openai.com/docs/guides/embeddings) to help you answer this question.

#### ! Answer #2:

The text-embedding-3-small has 1536 dimensions

</div>

### Finding the Embeddings for Our Chunks

First, let's find all our embeddings for each chunk and store them in a convenient format for later.

In [24]:
embeddings_dict = {}

for chunk in chunks:
  embeddings_dict[chunk] = embedding_model.embed_query(chunk)

In [25]:
for k,v in embeddings_dict.items():
  print(f"Chunk - {k}")
  print("---")
  print(f"Embedding - Vector of Size: {len(v)}")
  print("\n\n")

Chunk - Bases and Units
Groups of models that are selected within an army and which move and fight together are called Units. All the models within a Unit will be mounted on one or more Bases.
When the rules refer to a Unit, it means all the Bases within that Unit.
Unit Profiles
---
Embedding - Vector of Size: 1536



Chunk - Unit Profiles
All Units in Epic Warpath have a profile of statistics or ‘stats’ which show how effective the Bases in the Unit are at moving, shooting, assaulting and surviving in a combat zone. In addition, the height of the Bases in a Unit, their Keywords and weaponry are all included in the profile.
---
Embedding - Vector of Size: 1536



Chunk - Some Units may be so poor at doing something they will be effectively unable to do it at all! Such Units will have a stat value of “-“ where appropriate.
Some examples of Unit profiles are shown below. [EXAMPLE PROFILES, ANNOTATED]
Version PT v1
5
A Unit profile details the following information:
Unit Name: Each Unit i

Okay, great. Let's create a query - and then embed it!

In [26]:
query = "Can LCEL help take code from the notebook to production?"
query = "Summarize unit movement"

query_vector = embedding_model.embed_query(query)
print(f"Vector of Size: {len(query_vector)}")

Vector of Size: 1536


Now, let's compare it against each existing chunk's embedding by using cosine similarity.

In [27]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

In [28]:
max_similarity = -float('inf')
closest_chunk = ""

for chunk, chunk_vector in embeddings_dict.items():
  cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

  if cosine_similarity_score > max_similarity:
    closest_chunk = chunk
    max_similarity = cosine_similarity_score

print(closest_chunk)
print(max_similarity)

15
Moving Units
To move a Unit, simply measure the distance from the edge of the Bases and move them up to the maximum the Unit can move for the action being performed. This move can be in a straight line, or curved to avoid other models or terrain, and end up facing into any direction. Do make sure that the final positions of all the Bases in the Unit keeps it in Coherency and that no part of any base has moved more than the maximum distance allowed.
0.5514061995946075


And we get the expected result, which is the passage that specifically talking about troop movement

### Creating a Retriever

Now that we have an idea of how we're getting our most relevant information - let's see how we could create a pipeline that would automatically extract the closest chunk to our query and use it as context for our prompt!

First, we'll wrap the above in a helper function!

In [29]:
def retrieve_context(query, embeddings_dict, embedding_model):
  query_vector = embedding_model.embed_query(query)
  max_similarity = -float('inf')
  closest_chunk = ""

  for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
      closest_chunk = chunk
      max_similarity = cosine_similarity_score

  return closest_chunk

Now, let's add it to our pipeline!

In [30]:
def simple_rag(query, embeddings_dict, embedding_model, chat_chain):
  context = retrieve_context(query, embeddings_dict, embedding_model)

  response = chat_chain.invoke({"query" : query, "context" : context})

  return_package = {
      "query" : query,
      "response" : response,
      "retriever_context" : context
  }

  return return_package

In [34]:

simple_rag("How do units move?", embeddings_dict, embedding_model, chat_chain)

{'query': 'How do units move?',
 'response': AIMessage(content='To move a Unit, measure the distance from the edge of the Bases and move them up to the maximum distance the Unit can move for the action being performed. The movement can be in a straight line or curved to avoid other models or terrain, and the Unit can end up facing any direction. Ensure that the final positions of all the Bases in the Unit maintain Coherency and that no part of any base has moved more than the maximum distance allowed.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 154, 'total_tokens': 245}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_a2ff031fb5', 'finish_reason': 'stop', 'logprobs': None}, id='run-c50ea78e-f3d7-4261-8673-a01824a584d3-0', usage_metadata={'input_tokens': 154, 'output_tokens': 91, 'total_tokens': 245}),
 'retriever_context': '15\nMoving Units\nTo move a Unit, simply measure the distance from the edge of 

<div style="border: 2px solid white; background: black; padding: 10px;">

#### ❓ Question #3:

What does LCEL do that makes it more reliable at scale?

> HINT: Use your newly created `simple_rag` to help you answer this question!

#### ! Answer #3:

Per our chain:
    LCEL makes chains more reliable at scale by providing full support for sync,
    async, batch, and streaming operations. This comprehensive support enables easy
    prototyping and ensures that the same chain can be efficiently deployed in
    various environments, including those requiring asynchronous or streaming
    interfaces. This flexibility and robustness reduce the complexity and potential
    issues that can arise when scaling applications.

LCEL handles the challenges of scalability, complexity and reliability in building and deploying chains.
It minimizes the risk of performance bottlenecks when scaling up resulting in robust and responsive applications. 
It can be used to prototype complex chains but can also support scaling up to production environments.
Key features for scaling:
- Support for synchronous, asynchronous, batch and streaming operations 
- single interface with consistent behavior across all execution modes
- designed from the ground up to handle increased load and complexity as applications grow
- adaptable to web applications, microservices, IoT and edge computing

</div>

In [36]:

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""


chunks = text_splitter.split_text(CONTEXT)
embeddings_dict = {}

for chunk in chunks:
  embeddings_dict[chunk] = embedding_model.embed_query(chunk)

response = simple_rag("What does LCEL do that makes it more reliable at scale?", embeddings_dict, embedding_model, chat_chain)

import textwrap
def format_text(text, width=80):
    wrapped_text = textwrap.fill(text, width)
    print(wrapped_text)

format_text(response["response"].content)

LCEL makes chains more reliable at scale by automatically providing full sync,
async, batch, and streaming support. This ensures that any chain constructed
using LCEL can handle different execution models efficiently, making it easier
to scale and adapt to various use cases.
